In [6]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm

# Plotting and Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Language Detection packages
# `langdetect` for detecting language
from langdetect import detect as langdetect_detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# `langid` for an alternative language detection method
from langid import classify as langid_classify

# Text Preprocessing and NLP
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords

# Tokenizing sentences/words
from nltk.tokenize import word_tokenize

# Part-of-speech tagging
from nltk import pos_tag

# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer
import nltk

# Regular expressions for text pattern matching
import re

# Word Cloud generation
from wordcloud import WordCloud

In [1]:
import os
import sys
from pathlib import Path

if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("working dir:", workding_dir)

working dir: /home/inflaton/code/CrediNews


In [11]:
from datasets import load_dataset, concatenate_datasets, Dataset

datasets = load_dataset(
    "csv",
    data_files={
        "train": [
            "dataset/train_data_1.csv",
            "dataset/train_data_2.csv",
            "dataset/train_data_3.csv",
            "dataset/train_data_4.csv",
        ],
        "test": "dataset/test_data.csv",
        "rewritten_train": [
            "dataset/rewritten_train_data_1.csv",
            "dataset/rewritten_train_data_2.csv",
            "dataset/rewritten_train_data_3.csv",
            "dataset/rewritten_train_data_4.csv",
        ],
        "rewritten_test": "dataset/rewritten_test_data.csv",
    },
)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
    rewritten_train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    rewritten_test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
})

### RNN with Word2Vec (best RNN model)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import Word2Vec
import tensorflow as tf
import numpy as np
import random

from gensim.models import Word2Vec
import numpy as np


def train_word2vec_embeddings(
    train_texts, word_index, vocab_size=5000, embedding_dim=128
):
    """Tokenizes text, trains Word2Vec, and creates an embedding matrix."""
    sentences = [text.split() for text in train_texts]  # Convert text into word lists

    # Train Word2Vec
    word2vec_model = Word2Vec(
        sentences=sentences,
        vector_size=embedding_dim,
        window=5,
        min_count=1,
        workers=4,
        sg=1,
    )

    # Initialize embedding matrix
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    for word, i in word_index.items():
        if i < vocab_size and word in word2vec_model.wv:
            embedding_matrix[i] = word2vec_model.wv[word]

    return embedding_matrix


def create_rnn_model(
    vocab_size,
    embedding_matrix,
    input_length,
    rnn_units=64,
    dropout_rate=0.5,
    learning_rate=0.001,
):
    """Creates an RNN model using Word2Vec embeddings."""
    model = Sequential(
        [
            Embedding(
                vocab_size,
                embedding_matrix.shape[1],
                weights=[embedding_matrix],
                input_length=input_length,
                trainable=True,
            ),
            SimpleRNN(rnn_units, activation="tanh"),
            Dropout(dropout_rate),
            Dense(1, activation="sigmoid"),
        ]
    )

    model.compile(
        optimizer=Adam(learning_rate), loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model

2025-02-21 09:10:21.884386: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-21 09:10:22.039300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740100222.096444    1124 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740100222.113366    1124 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 09:10:22.267559: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split


def train_model(
    train_data,
    val_data,
    rnn_units=64,
    dropout_rate=0.5,
    epochs=10,
    batch_size=64,
    grid_search=False,
):
    """Trains an RNN model using Word2Vec embeddings."""
    seed = 42
    tf.random.set_seed(seed)
    np.random.seed(seed)

    vocab_size = 5000
    embedding_dim = 128
    max_sequence_length = 300

    print(f"\n🚀 Training RNN with rnn_units={rnn_units}, dropout_rate={dropout_rate}")

    # Extract texts and labels
    train_texts = train_data["processed_full_content"]
    val_texts = val_data["processed_full_content"]
    y_train = train_data["label"]
    y_val = val_data["label"]

    # Tokenize texts
    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(train_texts)

    # Convert texts to sequences and pad them
    X_train = pad_sequences(
        tokenizer.texts_to_sequences(train_texts), maxlen=max_sequence_length
    )
    X_val = pad_sequences(
        tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length
    )

    # Train Word2Vec and create embeddings
    embedding_matrix = train_word2vec_embeddings(
        train_texts, tokenizer.word_index, vocab_size, embedding_dim
    )

    # Create and compile the RNN model
    model = create_rnn_model(
        vocab_size=vocab_size,
        embedding_matrix=embedding_matrix,
        input_length=max_sequence_length,
        rnn_units=rnn_units,
        dropout_rate=dropout_rate,
    )

    # Early stopping
    early_stopping = EarlyStopping(
        monitor="val_loss", patience=3, restore_best_weights=True
    )

    # Train the model
    model.fit(
        X_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1,
    )

    # Evaluate model performance on validation data
    y_pred = (model.predict(X_val) > 0.5).astype(int).reshape(-1)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    #  Print evaluation metrics
    result = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    }
    print("\n🏆 Training Results:")

    for key, value in result.items():
        print(f"🔹 {key.capitalize()}: {value:.4f}")

    return result if grid_search else model

In [5]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
from tqdm import tqdm


def evaluate_model(model, train_data, val_data):
    print("Evaluating Model")

    max_words = 10000
    max_sequence_length = 300

    train_texts = train_data["processed_full_content"]
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(train_texts)

    y_val = val_data["label"]
    val_texts = val_data["processed_full_content"]

    X_val = pad_sequences(
        tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length
    )
    y_pred = (model.predict(X_val) > 0.5).astype(int)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1: {f1:.4f}")

In [7]:
from sklearn.model_selection import StratifiedKFold


def do_grid_search(data):
    """Performs Grid Search for the best RNN hyperparameters."""
    param_grid = {"rnn_units": [32, 64], "dropout_rate": [0.3, 0.5]}
    results = []
    best_score = 0
    best_params = None

    for rnn_units in param_grid["rnn_units"]:
        for dropout_rate in param_grid["dropout_rate"]:
            print(f"\n🔍 Testing rnn_units={rnn_units}, dropout_rate={dropout_rate}")

            kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            fold_scores = []

            for fold, (train_idx, val_idx) in enumerate(
                kfold.split(
                    data["processed_full_content"], data["label"].to_numpy().ravel()
                ),
                1,
            ):
                print(f"\n📂 Fold {fold}")

                train_data = data.iloc[train_idx]
                val_data = data.iloc[val_idx]

                result = train_model(
                    train_data,
                    val_data,
                    rnn_units=rnn_units,
                    dropout_rate=dropout_rate,
                    grid_search=True,
                )
                fold_scores.append(result["f1_score"])

            avg_score = np.mean(fold_scores)
            print(f"📊 Average F1-score: {avg_score:.4f}")

            if avg_score > best_score:
                best_score = avg_score
                best_params = {"rnn_units": rnn_units, "dropout_rate": dropout_rate}

    return best_params

In [12]:
train_data = datasets["train"].to_pandas()
val_data = datasets["test"].to_pandas()
data = pd.concat([train_data, val_data], ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60491 entries, 0 to 60490
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   label                   60491 non-null  int64 
 1   full_content            60491 non-null  object
 2   processed_full_content  60491 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [9]:
%%time
best_params = do_grid_search(data)


🔍 Testing rnn_units=32, dropout_rate=0.3

📂 Fold 1

🚀 Training RNN with rnn_units=32, dropout_rate=0.3


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1740053971.517864  477213 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 45525 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:01:00.0, compute capability: 8.9


Epoch 1/10


I0000 00:00:1740054029.613609  477569 service.cc:148] XLA service 0x7f1bb801b160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740054029.613987  477569 service.cc:156]   StreamExecutor device (0): NVIDIA RTX 6000 Ada Generation, Compute Capability 8.9
2025-02-20 20:20:29.649901: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1740054029.751974  477569 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/757 ━━━━━━━━━━━━━━━━━━━━ 37:56 3s/step - accuracy: 0.4062 - loss: 0.8048

...

757/757 ━━━━━━━━━━━━━━━━━━━━ 29s 37ms/step - accuracy: 0.8767 - loss: 0.2928 - val_accuracy: 0.9411 - val_loss: 0.1726
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 29s 38ms/step - accuracy: 0.9035 - loss: 0.2512 - val_accuracy: 0.9445 - val_loss: 0.1534
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 27s 36ms/step - accuracy: 0.9419 - loss: 0.1770 - val_accuracy: 0.9470 - val_loss: 0.1534
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 28s 37ms/step - accuracy: 0.9597 - loss: 0.1242 - val_accuracy: 0.8277 - val_loss: 0.5195
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 27s 36ms/step - accuracy: 0.9528 - loss: 0.1422 - val_accuracy: 0.9535 - val_loss: 0.1420
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 30s 40ms/step - accuracy: 0.9663 - loss: 0.1057 - val_accuracy: 0.9434 - val_loss: 0.1786
Epoch 7/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 29s 39ms/step - accuracy: 0.9646 - loss: 0.1089 - val_accuracy: 0.9335 - val_loss: 0.2006
Epoch 8/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 30s 39ms/step - accuracy: 0.9761 - loss: 0.0739 - val_accurac

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.8443 - loss: 0.3528 - val_accuracy: 0.9410 - val_loss: 0.1628
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 28s 37ms/step - accuracy: 0.9462 - loss: 0.1597 - val_accuracy: 0.9468 - val_loss: 0.1570
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 29s 38ms/step - accuracy: 0.9567 - loss: 0.1311 - val_accuracy: 0.9397 - val_loss: 0.1759
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 28s 37ms/step - accuracy: 0.9591 - loss: 0.1276 - val_accuracy: 0.9495 - val_loss: 0.1572
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 28s 37ms/step - accuracy: 0.9664 - loss: 0.1056 - val_accuracy: 0.9459 - val_loss: 0.1645
379/379 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step

🏆 Training Results:
🔹 Accuracy: 0.9468
🔹 Precision: 0.9678
🔹 Recall: 0.9085
🔹 F1_score: 0.9372

📂 Fold 5

🚀 Training RNN with rnn_units=64, dropout_rate=0.5
Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 29s 36ms/step - accuracy: 0.8571 - loss: 0.3174 - val_accuracy: 0.9403 - val_loss: 0.1676
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 31s 41ms/step - accuracy: 0.9475 - loss: 0.1517 - val_accuracy: 0.9390 - val_loss: 0.1735
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 33s 43ms/step - accuracy: 0.9603 - loss: 0.1166 - val_accuracy: 0.9011 - val_loss: 0.2458
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 32s 43ms/step - accuracy: 0.9326 - loss: 0.1866 - val_accuracy: 0.9449 - val_loss: 0.1682
379/379 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step

🏆 Training Results:
🔹 Accuracy: 0.9403
🔹 Precision: 0.9408
🔹 Recall: 0.9216
🔹 F1_score: 0.9311
📊 Average F1-score: 0.9419
CPU times: user 1h 41min 53s, sys: 20min 17s, total: 2h 2min 10s
Wall time: 1h 23min 22s


In [10]:
%%time
model = train_model(
    train_data,
    val_data,
    rnn_units=best_params["rnn_units"],
    dropout_rate=best_params["dropout_rate"],
)


🚀 Training RNN with rnn_units=64, dropout_rate=0.3
Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


851/851 ━━━━━━━━━━━━━━━━━━━━ 33s 38ms/step - accuracy: 0.8755 - loss: 0.2881 - val_accuracy: 0.9450 - val_loss: 0.1763
Epoch 2/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 30s 36ms/step - accuracy: 0.8983 - loss: 0.2596 - val_accuracy: 0.9458 - val_loss: 0.1523
Epoch 3/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 30s 35ms/step - accuracy: 0.8440 - loss: 0.3406 - val_accuracy: 0.9410 - val_loss: 0.1743
Epoch 4/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 31s 37ms/step - accuracy: 0.9086 - loss: 0.2363 - val_accuracy: 0.9458 - val_loss: 0.1668
Epoch 5/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 30s 35ms/step - accuracy: 0.9525 - loss: 0.1420 - val_accuracy: 0.9471 - val_loss: 0.1551
190/190 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step

🏆 Training Results:
🔹 Accuracy: 0.9458
🔹 Precision: 0.9431
🔹 Recall: 0.9324
🔹 F1_score: 0.9377
CPU times: user 5min 36s, sys: 1min 5s, total: 6min 41s
Wall time: 4min 4s


In [11]:
model.save("results/RNN_model_original.keras")

In [12]:
# load model
from tensorflow.keras.models import load_model

model2 = load_model("results/RNN_model_original.keras")
model2.summary()

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_20 (Embedding)        │ (None, 300, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_20 (SimpleRNN)       │ (None, 64)             │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,957,253 (7.47 MB)

 Trainable params: 652,417 (2.49 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,304,836 (4.98 MB)

In [13]:
%%time
evaluate_model(model, train_data, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step
Accuracy: 0.9428
Precision: 0.9492
Recall: 0.9184
F1: 0.9336
CPU times: user 8.63 s, sys: 3.26 s, total: 11.9 s
Wall time: 12.8 s


In [14]:
%%time
evaluate_model(model2, train_data, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step
Accuracy: 0.9428
Precision: 0.9492
Recall: 0.9184
F1: 0.9336
CPU times: user 8.95 s, sys: 1.81 s, total: 10.8 s
Wall time: 12.6 s


In [10]:
val_data_rewritten = datasets["rewritten_test"].to_pandas()
train_data_rewritten = datasets["rewritten_train"].to_pandas()
data_rewritten = pd.concat(
    [train_data, train_data_rewritten, val_data, val_data_rewritten], ignore_index=True
)
data_rewritten.info()

NameError: name 'train_data' is not defined

In [16]:
%%time

evaluate_model(model, train_data, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step
Accuracy: 0.7476
Precision: 0.8222
Recall: 0.5399
F1: 0.6518
CPU times: user 8.62 s, sys: 2.33 s, total: 10.9 s
Wall time: 12.6 s


In [17]:
%%time

evaluate_model(model2, train_data, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step
Accuracy: 0.7476
Precision: 0.8222
Recall: 0.5399
F1: 0.6518
CPU times: user 8.36 s, sys: 3.15 s, total: 11.5 s
Wall time: 12.4 s


In [18]:
%%time

best_params_rewritten = do_grid_search(data_rewritten)
best_params_rewritten


🔍 Testing rnn_units=32, dropout_rate=0.3

📂 Fold 1

🚀 Training RNN with rnn_units=32, dropout_rate=0.3
Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 50s 32ms/step - accuracy: 0.7651 - loss: 0.4858 - val_accuracy: 0.8893 - val_loss: 0.2961
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 51s 33ms/step - accuracy: 0.8557 - loss: 0.3523 - val_accuracy: 0.8897 - val_loss: 0.2829
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 48s 32ms/step - accuracy: 0.8482 - loss: 0.3651 - val_accuracy: 0.8579 - val_loss: 0.3296
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 49s 32ms/step - accuracy: 0.8860 - loss: 0.2858 - val_accuracy: 0.7471 - val_loss: 0.4992
Epoch 5/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 53s 35ms/step - accuracy: 0.8256 - loss: 0.3945 - val_accuracy: 0.8469 - val_loss: 0.3139
757/757 ━━━━━━━━━━━━━━━━━━━━ 25s 33ms/step

🏆 Training Results:
🔹 Accuracy: 0.8897
🔹 Precision: 0.9150
🔹 Recall: 0.8244
🔹 F1_score: 0.8673

📂 Fold 2

🚀 Training RNN with rnn_units=32, dropout_rate=0.3
Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 50s 33ms/step - accuracy: 0.8014 - loss: 0.4309 - val_accuracy: 0.8612 - val_loss: 0.3659
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 51s 33ms/step - accuracy: 0.8126 - loss: 0.4210 - val_accuracy: 0.8885 - val_loss: 0.3054
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 49s 32ms/step - accuracy: 0.8621 - loss: 0.3490 - val_accuracy: 0.8428 - val_loss: 0.3574
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 49s 33ms/step - accuracy: 0.8862 - loss: 0.2966 - val_accuracy: 0.8853 - val_loss: 0.2860
Epoch 5/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 50s 33ms/step - accuracy: 0.8955 - loss: 0.2676 - val_accuracy: 0.8994 - val_loss: 0.2669
Epoch 6/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 52s 35ms/step - accuracy: 0.8883 - loss: 0.2879 - val_accuracy: 0.7937 - val_loss: 0.4373
Epoch 7/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 52s 34ms/step - accuracy: 0.8371 - loss: 0.3777 - val_accuracy: 0.8404 - val_loss: 0.3535
Epoch 8/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 51s 34ms/step - accuracy: 0.8774 - loss: 0.29

...

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 64s 42ms/step - accuracy: 0.8214 - loss: 0.4215 - val_accuracy: 0.8703 - val_loss: 0.3375
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 63s 42ms/step - accuracy: 0.8609 - loss: 0.3590 - val_accuracy: 0.6267 - val_loss: 0.7126
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 64s 42ms/step - accuracy: 0.8006 - loss: 0.4355 - val_accuracy: 0.8545 - val_loss: 0.3667
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 62s 41ms/step - accuracy: 0.8335 - loss: 0.3826 - val_accuracy: 0.8774 - val_loss: 0.3035
Epoch 5/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 66s 44ms/step - accuracy: 0.8674 - loss: 0.3245 - val_accuracy: 0.7265 - val_loss: 0.5197
Epoch 6/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 68s 45ms/step - accuracy: 0.8195 - loss: 0.4061 - val_accuracy: 0.8606 - val_loss: 0.3587
Epoch 7/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 68s 45ms/step - accuracy: 0.8771 - loss: 0.3128 - val_accuracy: 0.8859 - val_loss: 0.3171
757/757 ━━━━━━━━━━━━━━━━━━━━ 29s 38ms/step

🏆 Training Results:
🔹 Accuracy: 0.8774
🔹 Pr

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 67s 43ms/step - accuracy: 0.8054 - loss: 0.4414 - val_accuracy: 0.7964 - val_loss: 0.4228
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 66s 44ms/step - accuracy: 0.8176 - loss: 0.4089 - val_accuracy: 0.7462 - val_loss: 0.5200
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 62s 41ms/step - accuracy: 0.8316 - loss: 0.3953 - val_accuracy: 0.6835 - val_loss: 0.5544
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 73s 48ms/step - accuracy: 0.7708 - loss: 0.4629 - val_accuracy: 0.8843 - val_loss: 0.2879
Epoch 5/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 60s 39ms/step - accuracy: 0.8701 - loss: 0.3240 - val_accuracy: 0.8677 - val_loss: 0.3675
Epoch 6/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 74s 49ms/step - accuracy: 0.8327 - loss: 0.3894 - val_accuracy: 0.8311 - val_loss: 0.3872
Epoch 7/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 68s 45ms/step - accuracy: 0.8676 - loss: 0.3311 - val_accuracy: 0.8818 - val_loss: 0.3063
757/757 ━━━━━━━━━━━━━━━━━━━━ 25s 33ms/step

🏆 Training Results:
🔹 Accuracy: 0.8843
🔹 Pr

{'rnn_units': 32, 'dropout_rate': 0.3}

In [19]:
%%time

train_data_combined = pd.concat([train_data, train_data_rewritten], ignore_index=True)
val_data_combined = pd.concat([val_data, val_data_rewritten], ignore_index=True)
model_combined = train_model(
    train_data_combined,
    val_data_combined,
    rnn_units=best_params_rewritten["rnn_units"],
    dropout_rate=best_params_rewritten["dropout_rate"],
)
model_combined.save("results/RNN_model_combined.keras")


🚀 Training RNN with rnn_units=32, dropout_rate=0.3
Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1702/1702 ━━━━━━━━━━━━━━━━━━━━ 62s 36ms/step - accuracy: 0.7540 - loss: 0.5081 - val_accuracy: 0.8737 - val_loss: 0.3421
Epoch 2/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 60s 35ms/step - accuracy: 0.8478 - loss: 0.3807 - val_accuracy: 0.8814 - val_loss: 0.3203
Epoch 3/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 51s 30ms/step - accuracy: 0.8550 - loss: 0.3554 - val_accuracy: 0.8807 - val_loss: 0.3291
Epoch 4/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 56s 33ms/step - accuracy: 0.8766 - loss: 0.3201 - val_accuracy: 0.7088 - val_loss: 0.5368
Epoch 5/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 59s 35ms/step - accuracy: 0.8562 - loss: 0.3462 - val_accuracy: 0.8691 - val_loss: 0.3031
Epoch 6/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 61s 36ms/step - accuracy: 0.8900 - loss: 0.2753 - val_accuracy: 0.8904 - val_loss: 0.2817
Epoch 7/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 58s 34ms/step - accuracy: 0.8822 - loss: 0.3053 - val_accuracy: 0.8774 - val_loss: 0.3367
Epoch 8/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 55s 32ms/step - accuracy: 0.8848 - loss: 0.31

In [20]:
evaluate_model(model_combined, train_data_combined, val_data_combined)

Evaluating Model
379/379 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step
Accuracy: 0.8800
Precision: 0.8245
Recall: 0.9220
F1: 0.8705


In [21]:
evaluate_model(model_combined, train_data_combined, val_data)

Evaluating Model


190/190 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step
Accuracy: 0.9367
Precision: 0.9081
Recall: 0.9516
F1: 0.9293


In [22]:
evaluate_model(model_combined, train_data_combined, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step
Accuracy: 0.8233
Precision: 0.7508
Recall: 0.8923
F1: 0.8155


In [23]:
model_combined2 = load_model("results/RNN_model_combined.keras")
model_combined2.summary()

Model: "sequential_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_41 (Embedding)        │ (None, 300, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_41 (SimpleRNN)       │ (None, 32)             │         5,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,935,557 (7.38 MB)

 Trainable params: 645,185 (2.46 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,290,372 (4.92 MB)

In [24]:
evaluate_model(model_combined2, train_data_combined, val_data_combined)

Evaluating Model
379/379 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step
Accuracy: 0.8800
Precision: 0.8245
Recall: 0.9220
F1: 0.8705


In [25]:
evaluate_model(model_combined2, train_data_combined, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step
Accuracy: 0.9367
Precision: 0.9081
Recall: 0.9516
F1: 0.9293


In [26]:
evaluate_model(model_combined2, train_data_combined, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step
Accuracy: 0.8233
Precision: 0.7508
Recall: 0.8923
F1: 0.8155


In [27]:
data_rewritten = pd.concat(
    [train_data_rewritten, val_data_rewritten], ignore_index=True
)
data_rewritten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60491 entries, 0 to 60490
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   label                   60491 non-null  int64 
 1   full_content            60491 non-null  object
 2   processed_full_content  60491 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [28]:
%%time

best_params_rewritten = do_grid_search(data_rewritten)
best_params_rewritten


🔍 Testing rnn_units=32, dropout_rate=0.3

📂 Fold 1

🚀 Training RNN with rnn_units=32, dropout_rate=0.3
Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 30s 39ms/step - accuracy: 0.7117 - loss: 0.5622 - val_accuracy: 0.8197 - val_loss: 0.4129
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 31s 40ms/step - accuracy: 0.8100 - loss: 0.4366 - val_accuracy: 0.8206 - val_loss: 0.4314
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 27s 35ms/step - accuracy: 0.8441 - loss: 0.3917 - val_accuracy: 0.4648 - val_loss: 1.0013
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 25s 33ms/step - accuracy: 0.6774 - loss: 0.6000 - val_accuracy: 0.7522 - val_loss: 0.5235
379/379 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step

🏆 Training Results:
🔹 Accuracy: 0.8197
🔹 Precision: 0.7526
🔹 Recall: 0.8759
🔹 F1_score: 0.8096

📂 Fold 2

🚀 Training RNN with rnn_units=32, dropout_rate=0.3
Epoch 1/10


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 29s 37ms/step - accuracy: 0.7131 - loss: 0.5646 - val_accuracy: 0.8136 - val_loss: 0.4314
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 28s 37ms/step - accuracy: 0.7853 - loss: 0.4778 - val_accuracy: 0.8270 - val_loss: 0.4341
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 29s 38ms/step - accuracy: 0.7984 - loss: 0.4800 - val_accuracy: 0.8145 - val_loss: 0.4221
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 28s 36ms/step - accuracy: 0.8349 - loss: 0.4019 - val_accuracy: 0.8084 - val_loss: 0.4507
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 27s 35ms/step - accuracy: 0.8491 - loss: 0.3732 - val_accuracy: 0.7321 - val_loss: 0.5749
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 25s 34ms/step - accuracy: 0.8547 - loss: 0.3674 - val_accuracy: 0.8330 - val_loss: 0.4319
379/379 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step

🏆 Training Results:
🔹 Accuracy: 0.8145
🔹 Precision: 0.8294
🔹 Recall: 0.7251
🔹 F1_score: 0.7737

📂 Fold 3

🚀 Training RNN with rnn_units=32, dropout_rate=0.3
Epoch 1/10


...

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 40s 51ms/step - accuracy: 0.6833 - loss: 0.5937 - val_accuracy: 0.6635 - val_loss: 0.6120
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 34s 45ms/step - accuracy: 0.7294 - loss: 0.5479 - val_accuracy: 0.7791 - val_loss: 0.4854
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 32s 42ms/step - accuracy: 0.7659 - loss: 0.5076 - val_accuracy: 0.7163 - val_loss: 0.5570
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 36s 48ms/step - accuracy: 0.7166 - loss: 0.5527 - val_accuracy: 0.8266 - val_loss: 0.4226
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 37s 48ms/step - accuracy: 0.8108 - loss: 0.4424 - val_accuracy: 0.8214 - val_loss: 0.4376
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 27s 36ms/step - accuracy: 0.7840 - loss: 0.4826 - val_accuracy: 0.6814 - val_loss: 0.5773
Epoch 7/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 35s 46ms/step - accuracy: 0.7258 - loss: 0.5388 - val_accuracy: 0.8266 - val_loss: 0.4395
379/379 ━━━━━━━━━━━━━━━━━━━━ 16s 42ms/step

🏆 Training Results:
🔹 Accuracy: 0.8266
🔹 Precision: 0.827

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 39s 51ms/step - accuracy: 0.7178 - loss: 0.5652 - val_accuracy: 0.8264 - val_loss: 0.4154
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 37s 49ms/step - accuracy: 0.8085 - loss: 0.4547 - val_accuracy: 0.6484 - val_loss: 0.6221
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 39s 51ms/step - accuracy: 0.7533 - loss: 0.5169 - val_accuracy: 0.8139 - val_loss: 0.4343
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 36s 48ms/step - accuracy: 0.8002 - loss: 0.4493 - val_accuracy: 0.6788 - val_loss: 0.5872
379/379 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step

🏆 Training Results:
🔹 Accuracy: 0.8264
🔹 Precision: 0.8073
🔹 Recall: 0.7923
🔹 F1_score: 0.7997
📊 Average F1-score: 0.7989
CPU times: user 1h 14min 45s, sys: 23min 24s, total: 1h 38min 10s
Wall time: 1h 22min 35s


{'rnn_units': 32, 'dropout_rate': 0.5}

In [13]:
%%time


model_rewritten = train_model(
    train_data_rewritten,
    val_data_rewritten,
    rnn_units=best_params_rewritten["rnn_units"],
    dropout_rate=best_params_rewritten["dropout_rate"],
)
model_rewritten.save("results/RNN_model_rewritten.keras")


🚀 Training RNN with rnn_units=32, dropout_rate=0.5


/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1740100502.123699    1124 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 45689 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:01:00.0, compute capability: 8.9


Epoch 1/10


I0000 00:00:1740100504.135575    2322 service.cc:148] XLA service 0x7efd6400a9e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740100504.135846    2322 service.cc:156]   StreamExecutor device (0): NVIDIA RTX 6000 Ada Generation, Compute Capability 8.9
2025-02-21 09:15:04.162190: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1740100504.247861    2322 cuda_dnn.cc:529] Loaded cuDNN version 90300


  4/851 ━━━━━━━━━━━━━━━━━━━━ 25s 30ms/step - accuracy: 0.4515 - loss: 0.8570

I0000 00:00:1740100504.716820    2322 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


851/851 ━━━━━━━━━━━━━━━━━━━━ 25s 29ms/step - accuracy: 0.7100 - loss: 0.5744 - val_accuracy: 0.7944 - val_loss: 0.4637
Epoch 2/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 25s 29ms/step - accuracy: 0.7463 - loss: 0.5310 - val_accuracy: 0.8207 - val_loss: 0.4364
Epoch 3/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 22s 26ms/step - accuracy: 0.7215 - loss: 0.5539 - val_accuracy: 0.8136 - val_loss: 0.4865
Epoch 4/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 24s 28ms/step - accuracy: 0.7745 - loss: 0.5199 - val_accuracy: 0.7898 - val_loss: 0.4899
Epoch 5/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 24s 29ms/step - accuracy: 0.7297 - loss: 0.5610 - val_accuracy: 0.7193 - val_loss: 0.5526
190/190 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step

🏆 Training Results:
🔹 Accuracy: 0.8207
🔹 Precision: 0.8612
🔹 Recall: 0.7034
🔹 F1_score: 0.7744
CPU times: user 4min 13s, sys: 37.5 s, total: 4min 51s
Wall time: 3min 7s


In [16]:
evaluate_model(model_rewritten, train_data_rewritten, val_data_combined)

Evaluating Model
379/379 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step
Accuracy: 0.8617
Precision: 0.8835
Recall: 0.7879
F1: 0.8330


In [17]:
evaluate_model(model_rewritten, train_data_rewritten, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Accuracy: 0.8231
Precision: 0.8466
Recall: 0.7276
F1: 0.7826


In [18]:
evaluate_model(model_rewritten, train_data_rewritten, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step
Accuracy: 0.9003
Precision: 0.9178
Recall: 0.8481
F1: 0.8816
